In [10]:
%pylab
%matplotlib inline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()#导入iris数据集

data = iris.data
data = data[:100]
labels = iris.target
labels = labels[:100]
feature_name = iris.feature_names
print("feature_name", feature_name)

#我们选择了sepal length (cm)', 'sepal width (cm)'，并且更改了labels
data = data[:, 0:2]
labels[labels == 0] = -1
feature_name = feature_name[:2]

X_train, X_test, y_train, y_test = train_test_split(data, labels)#划分数据集

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
feature_name ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [37]:
class my_svm(object):
    def __init__(self, max_itr = 100, kernel_name = 'linear'):
        self.max_itr = max_itr
        self.kernel_name = kernel_name

    def input_pramas(self, x, y, C = 1 ):
        self.x = x
        self.y = y
        self.b = 0
        self.C = C
        self.dimention = len(x)

        self.alp = np.zeros(self.dimention)
        self.E = np.array([self.E_function(i) for i in range(self.dimention)])

    def E_function(self, i):
        return self.g_function(i) - self.y[i]

    def g_function(self, i):
        temp = self.b
        for j in range(self.dimention):
            temp += self.alp[j]*self.y[i]*self.kernel(self.x[i], self.x[j])
        return temp

    def kernel(self, xi, xj):
        if self.kernel_name == 'linear':
            return np.dot(xi, xj)
        elif self.kernel_name == 'mutinomial':
            return (np.dot(xi, xj))**2
        else:
            return 0

    def lr_function(self, i, j):
        return self.kernel(i, i) + self.kernel(j, j) - 2*self.kernel(i, j)

    def KKT(self, i):
        if 0 < self.alp[i] < self.C:
            return self.y[i]*self.g_function(i) == 1
        elif self.alp[i] == self.C:
            return self.y[i]*self.g_function(i) <= 1
        elif self.alp[i] == 0:
            return self.y[i]*self.g_function(i) >= 1
        else:
            return False

    def clip(self, alp_new_unc, L, H):
        if alp_new_unc <= L:
            return L
        elif alp_new_unc >= H:
            return H
        else:
            return alp_new_unc

    def fit(self, x, y, C = 1):
        self.input_pramas(x, y, C)
        #print(self.alp)
        for itr in range(self.max_itr):
            for i in range(self.dimention):
                if not self.KKT(i):
                    alp1 = self.alp[i]
                    E1 = self.E[i]
                    y1 = self.y[i]

                    j = np.argmax(np.abs((self.E - E1)))
                    alp2 = self.E[j]
                    E2 = self.E[j]
                    y2 = self.y[j]

                    lr = self.lr_function(i, j)
                    #print(i)

                    alp2_new_unc = alp2 + self.y[j]*(self.E[j] - self.E[i]) / lr

                    C = self.C
                    if y[i] == y[j]:
                        L = max(0, alp1+alp2 - C)
                        H = min(alp1+alp2, C)
                    else:
                        L = max(0, alp2 - alp1)
                        H = min(alp2 - alp1 + C, C)

                    alp2_new = self.clip(alp2_new_unc, L, H)
                    alp1_new = alp1 + y1*y2*(alp2 - alp2_new)
                    
                    self.alp[i] = alp1_new
                    self.alp[j] = alp2_new

                    b1_new = -E1 + y1*self.kernel(i, i)*(alp1 - alp1_new) + y2*self.kernel(i, j)*(alp2 - alp2_new) + self.b
                    b2_new = -E2 + y1*self.kernel(i, j)*(alp1 - alp1_new) + y2*self.kernel(j, j)*(alp2 - alp2_new) + self.b

                    if 0 < alp1_new < C:
                        b_new = b1_new
                    elif 0 < alp2_new < C:
                        b_new = b2_new
                    else:
                        b_new = (b1_new + b2_new) / 2

                    self.b = b_new

                    self.E[i] = self.g_function(i) - self.y[i]
                    self.E[j] = self.g_function(j) - self.y[j]

    def predict(self, x):
        res = np.zeros(len(x))
        for j in range(len(x)):
            temp = self.b
            for i in range(self.dimention):
                temp += self.alp[i]*self.y[i]*self.kernel(i, j)
            res[j] = 1 if temp > 0 else -1
        return res

In [38]:
pp = my_svm()
pp.fit(X_train, y_train)

C:\Users\hichens\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in double_scalars
C:\Users\hichens\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\hichens\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in double_scalars


In [39]:
y_predict = pp.predict(X_test)

In [40]:
y_predict

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [41]:
pp.score(X_test, y_test)

AttributeError: 'my_svm' object has no attribute 'score'

In [ ]:
pp.plot(data, labels, feature_name)

In [ ]:
from sklearn.linear_model import Perceptron
pp = Perceptron()
pp.fit(X_train, y_train)
y_pred = pp.predict(X_test)

In [ ]:
pp.score(X_test, y_test)

In [ ]:
x_ponits = np.arange(4, 8)
y_ = -(pp.coef_[0][0]*x_ponits + pp.intercept_)/pp.coef_[0][1]

plt.plot(x_ponits, y_)

plt.scatter(data[labels == -1][:, 0], data[labels == -1][:, 1])
plt.scatter(data[labels == 1][:, 0], data[labels == 1][:, 1])
plt.xlabel('sepal length')
plt.ylabel('sepal width')
plt.legend()

In [ ]:
clf.coef_